### Pandas Part 2
#### Date: 2025-12-01
**Agenda**:
> 1. Apply, lambda, map, transform 
> 2. Indexs 
> 3. Fill and drop NaN
> 4. Data types

**Materials**:
> [Seaborn](https://seaborn.pydata.org/tutorial/introduction.html) \
> [Pandas](https://www.w3schools.com/python/pandas/pandas_intro.asp) \
> [Kaggle](https://www.kaggle.com)

#### Install Library!!!!
##### or using conda
> **pip install pandas** \
**pip install matplotlib** \
**pip install seaborn**

In [1]:
import pandas as pd 
import numpy as np 

import seaborn as sns

In [2]:
import warnings

warnings.simplefilter(action='ignore', category=FutureWarning)

---
#### Об’єднання даних у Pandas: CONCAT, MERGE, JOIN

**pd.concat()** — об’єднання DataFrame один під одним аналог UNION ALL з SQL \

Використовується, коли потрібно:
- додати нові рядки (вертикально)
- додати нові стовпці (горизонтально)
- поєднати кілька DataFrame в один
- зібрати дані з різних частин або періодів часу

**Best Practise**:
> ignore_index=True, якщо старий індекс не потрібен \
> перед concat рекомендовано робити reset_index(drop=True) \
> потрібно стежити за однаковою схемою таблиць \
> для великих наборів даних роби concat пакетами ( batch)


In [3]:

jan = pd.DataFrame({
    "date": ["2024-01-01", "2024-01-02"],
    "sales": [100, 120]
})

feb = pd.DataFrame({
    "date": ["2024-02-01", "2024-02-02"],
    "sales": [130, 110]
})

df = pd.concat([jan, feb], ignore_index=True)
print(df)


         date  sales
0  2024-01-01    100
1  2024-01-02    120
2  2024-02-01    130
3  2024-02-02    110


In [4]:
# load dataset
tips = sns.load_dataset("tips")
penguins = sns.load_dataset("penguins")
flights = sns.load_dataset("flights")
titanic = sns.load_dataset("titanic")

In [5]:
tips1 = tips.iloc[:100]
tips2 = tips.iloc[100:]


In [ ]:
df = pd.concat([tips1, tips2], ignore_index=True)

In [6]:
df = pd.concat(
    [tips.assign(dataset="tips"),
     penguins.assign(dataset="penguins")],
    ignore_index=True
)

df.head()


,total_bill,tip,sex,smoker,day,time,size,dataset,species,island,bill_length_mm,bill_depth_mm,flipper_length_mm,body_mass_g
0,16.99,1.01,Female,No,Sun,Dinner,2.0,tips,NaN,NaN,NaN,NaN,NaN,NaN
1,10.34,1.66,Male,No,Sun,Dinner,3.0,tips,NaN,NaN,NaN,NaN,NaN,NaN
2,21.01,3.50,Male,No,Sun,Dinner,3.0,tips,NaN,NaN,NaN,NaN,NaN,NaN
3,23.68,3.31,Male,No,Sun,Dinner,2.0,tips,NaN,NaN,NaN,NaN,NaN,NaN
4,24.59,3.61,Female,No,Sun,Dinner,4.0,tips,NaN,NaN,NaN,NaN,NaN,NaN


In [7]:
left = tips.iloc[:10, :3]   # total_bill, tip, sex
right = tips.iloc[:10, 3:]  # smoker, day, time, size

df = pd.concat([left, right], axis=1)
df.head()


,total_bill,tip,sex,smoker,day,time,size
0,16.99,1.01,Female,No,Sun,Dinner,2
1,10.34,1.66,Male,No,Sun,Dinner,3
2,21.01,3.50,Male,No,Sun,Dinner,3
3,23.68,3.31,Male,No,Sun,Dinner,2
4,24.59,3.61,Female,No,Sun,Dinner,4


In [8]:
df = pd.concat(
    [tips.head(), penguins.head()],
    keys=["tips", "penguins"]
)

df


total_bill   tip     sex smoker  day    time  size species  \
tips     0       16.99  1.01  Female     No  Sun  Dinner   2.0     NaN   
         1       10.34  1.66    Male     No  Sun  Dinner   3.0     NaN   
         2       21.01  3.50    Male     No  Sun  Dinner   3.0     NaN   
         3       23.68  3.31    Male     No  Sun  Dinner   2.0     NaN   
         4       24.59  3.61  Female     No  Sun  Dinner   4.0     NaN   
penguins 0         NaN   NaN    Male    NaN  NaN     NaN   NaN  Adelie   
         1         NaN   NaN  Female    NaN  NaN     NaN   NaN  Adelie   
         2         NaN   NaN  Female    NaN  NaN     NaN   NaN  Adelie   
         3         NaN   NaN     NaN    NaN  NaN     NaN   NaN  Adelie   
         4         NaN   NaN  Female    NaN  NaN     NaN   NaN  Adelie   

               island  bill_length_mm  bill_depth_mm  flipper_length_mm  \
tips     0        NaN             NaN            NaN                NaN   
         1        NaN             NaN            NaN                NaN   
         2        NaN             NaN            NaN                NaN   
         3        NaN             NaN            NaN                NaN   
         4        NaN             NaN            NaN                NaN   
penguins 0  Torgersen            39.1           18.7              181.0   
         1  Torgersen            39.5           17.4              186.0   
         2  Torgersen            40.3           18.0              195.0   
         3  Torgersen             NaN            NaN                NaN   
         4  Torgersen            36.7           19.3              193.0   

            body_mass_g  
tips     0          NaN  
         1          NaN  
         2          NaN  
         3          NaN  
         4          NaN  
penguins 0       3750.0  
         1       3800.0  
         2       3250.0  
         3          NaN  
         4       3450.0

---
#### MERGE — це механізм горизонтального об’єднання двох таблиць за ключем (або декількома ключами), аналог SQL JOIN.
**Він дозволяє:**
- знаходити відповідні записи між таблицями
- додавати нові змінні з іншої таблиці
- синхронізувати дві різні системи даних

**Ключ** — це стовпець/стовпці, за якими можна однозначно (або майже однозначно) знайти відповідний запис.

**Ключ повинен:**
- мати однаковий зміст у двох таблицях
- мати однаковий тип (int, str, category)
- не містити зайвих пробілів / регістрів
- унікальним

**Ключ може бути:**
- однією колонкою
- кількома колонками
- індексом

Якщо ключів декілька рядки вважаються відповідними, коли всі ключі збігаються.

| Тип       | Опис                                                     | Коли використовується                  |
| --------- | -------------------------------------------------------- | -------------------------------------- |
| **left**  | Беремо всі рядки з лівої таблиці, додаємо збіги з правої | аналітика, побудова основних датасетів |
| **inner** | Беремо *тільки* збіги з обох таблиць                     | перетин даних                          |
| **right** | Все з правої, збіги з лівої                              | рідко                                  |
| **outer** | ВСІ рядки з обох таблиць, заповнюємо пропуски            | об’єднання систем      |


In [9]:
tips = sns.load_dataset("tips")

avg_by_sex = tips.groupby("sex")["tip"].mean().reset_index()
avg_by_sex.columns = ["sex", "avg_tip_by_sex"]

# merge
df = tips.merge(avg_by_sex, on="sex", how="left")
df.head()


,total_bill,tip,sex,smoker,day,time,size,avg_tip_by_sex
0,16.99,1.01,Female,No,Sun,Dinner,2,2.833448
1,10.34,1.66,Male,No,Sun,Dinner,3,3.089618
2,21.01,3.50,Male,No,Sun,Dinner,3,3.089618
3,23.68,3.31,Male,No,Sun,Dinner,2,3.089618
4,24.59,3.61,Female,No,Sun,Dinner,4,2.833448


In [10]:
sex_map = pd.DataFrame({
    "sex_english": ["Male", "Female"],
    "sex_ukr": ["чоловік", "жінка"]
})

df = tips.merge(sex_map, left_on="sex", right_on="sex_english", how="left")
df.head()


,total_bill,tip,sex,smoker,day,time,size,sex_english,sex_ukr
0,16.99,1.01,Female,No,Sun,Dinner,2,Female,жінка
1,10.34,1.66,Male,No,Sun,Dinner,3,Male,чоловік
2,21.01,3.50,Male,No,Sun,Dinner,3,Male,чоловік
3,23.68,3.31,Male,No,Sun,Dinner,2,Male,чоловік
4,24.59,3.61,Female,No,Sun,Dinner,4,Female,жінка


In [11]:
flights = sns.load_dataset("flights")

months = pd.DataFrame({
    "month": flights["month"].unique(),
    "season": ["winter","winter","spring","spring","spring",
               "summer","summer","summer","autumn","autumn","autumn","winter"]
})

df = flights.merge(months, on="month", how="left")
df.head()


,year,month,passengers,season
0,1949,Jan,112,winter
1,1949,Feb,118,winter
2,1949,Mar,132,spring
3,1949,Apr,129,spring
4,1949,May,121,spring


In [12]:
titanic = sns.load_dataset("titanic")

missing_age = (
    titanic.merge(
        titanic[titanic["age"].notna()][["age"]],
        left_index=True,
        right_index=True,
        how="left",
        indicator=True
    )
    .query('_merge == "left_only"')
)

missing_age.head()


,survived,pclass,sex,age_x,sibsp,parch,fare,embarked,class,who,adult_male,deck,embark_town,alive,alone,age_y,_merge
5,0,3,male,NaN,0,0,8.4583,Q,Third,man,True,NaN,Queenstown,no,True,NaN,left_only
17,1,2,male,NaN,0,0,13.0000,S,Second,man,True,NaN,Southampton,yes,True,NaN,left_only
19,1,3,female,NaN,0,0,7.2250,C,Third,woman,False,NaN,Cherbourg,yes,True,NaN,left_only
26,0,3,male,NaN,0,0,7.2250,C,Third,man,True,NaN,Cherbourg,no,True,NaN,left_only
28,1,3,female,NaN,0,0,7.8792,Q,Third,woman,False,NaN,Queenstown,yes,True,NaN,left_only


In [13]:
tips_idx = tips.set_index("day")
avg_day = tips.groupby("day")["tip"].mean()

df = tips_idx.merge(avg_day, left_index=True, right_index=True)


In [14]:
avg_bill = tips.groupby("day")["total_bill"].mean().reset_index()
avg_bill.columns = ["day", "avg_bill"]

df = tips.merge(avg_bill, on="day", how="left")


--- 
#### JOIN 
Це спеціальний спосіб злиття таблиць у pandas, який: 
> за замовчуванням робить LEFT JOIN \
> працює по індексу \
> дозволяє вказати колонку лівої таблиці (on=), але праву таблицю завжди шукає по індексу


##### Join vs Merge
| Перевага JOIN              | Чому важливо                |
| -------------------------- | --------------------------- |
| Лаконічний синтаксис       | менше коду, чистий пайплайн |
| Швидший                    | індекс працює як хеш-карта  |
| Ідеальний для агрегатів    | у 95% аналітичних задач     |
| Підтримує списки таблиць   | df.join([a,b,c])            |
| Читається як SQL LEFT JOIN | інтуїтивно                  |


In [ ]:
avg_by_day = tips.groupby("day")["tip"].mean().to_frame("avg_tip_by_day")
avg_by_day


In [ ]:
df = tips.set_index("day").join(avg_by_day).reset_index()
df.head()


In [ ]:
avg_by_sex = tips.groupby("sex")["tip"].mean().to_frame("avg_tip_by_sex")
df = tips.join(avg_by_sex, on="sex")
df.head()


In [ ]:
avg_by_time = tips.groupby("time")["tip"].mean().to_frame("avg_tip_by_time")
avg_by_size = tips.groupby("size")["tip"].mean().to_frame("avg_tip_by_size")


In [ ]:
df = (
    tips
    .join(avg_by_sex, on="sex")
    .join(avg_by_time, on="time")
    .join(avg_by_size, on="size")
)

df.head()


---
#### Пропуски
Пропуски у pandas позначаються як:
> NaN (Not a Number) — числові пропуски  \
> None — пропуски у текстових колонках \
> NaT — пропуски у датах \
> np.nan — альтернативне позначення



**Пропуски:**
> ламають агрегації \
> ускладнюють merge/join \
> дають некоректні середні значення \
> псують ML-моделі \
> спотворюють розподіли



#### Strategy
**Видалення пропусків (Drop)**
> пропусків дуже мало (<1%) \
> пропуск у цій колонці не критичний \
> неможливо коректно відновити значення

**Методи:**
> df.dropna() \
> df.dropna(subset=[...])


> **Заповнення константою (Constant Imputation)** \
> **Медіана/середнє (Median / Mean Imputation)** \
> **Мода (Mode Imputation)** \
> **Групові статистики (Group-Based Imputation)** \
> **Forward / Backward Fill (ffill/bfill)** \
> **Інтерполяція (Interpolate)** \
> **Predictive Imputation**

| Тип даних       | Краща стратегія       | Чому                     |
| --------------- | --------------------- | ------------------------ |
| Числові         | median / group-median | стійкість                |
| Категорії       | mode / “unknown”      | просто і надійно         |
| Дати            | interpolate / ffill   | зберігає часовий порядок |
| ML-фічі         | group stats           | найякісніше              |
| Дані з трендом  | interpolate           | відновлює форму          |
| Сегменти/класи  | “unknown”             | уникає змішування груп   |
| Вибірки для A/B | видалення             | не спотворює метрики     |


In [ ]:
titanic["age_filled"] = titanic["age"].fillna(titanic["age"].median())

In [ ]:
titanic["embarked_filled"] = titanic["embarked"].fillna(
    titanic["embarked"].mode()[0]
)


In [ ]:
titanic["age_group_filled"] = (
    titanic
    .groupby(["sex", "class"])["age"]
    .transform(lambda x: x.fillna(x.median()))
)


In [ ]:
flights = sns.load_dataset("flights")
flights["date"] = pd.to_datetime(flights["year"].astype(str) + "-" + flights["month"])
flights = flights.set_index("date")

flights_ffill = flights.ffill()


In [ ]:
flights_interp = flights.interpolate(method="linear")

In [ ]:
tips["tip_filled"] = (
    tips
    .groupby("day")["tip"]
    .transform(lambda x: x.fillna(x.median()))
)
